In [3]:
import tkinter as tk
from tkinter import messagebox
import firebase_admin
from firebase_admin import credentials, auth, firestore

# Path to the service account key file
service_account_path = "quizly.json"

# Initialize Firebase
cred = credentials.Certificate(service_account_path)
firebase_admin.initialize_app(cred)
db = firestore.client()

class QuizApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Quiz Application")
        self.user_type = None
        self.create_login_page()
    
    def create_login_page(self):
        self.clear_frame()
        self.login_frame = tk.Frame(self.master)
        self.login_frame.pack(pady=20)

        tk.Label(self.login_frame, text="Login", font=("Helvetica", 16)).grid(row=0, column=1, columnspan=2, pady=10)

        tk.Label(self.login_frame, text="Email:").grid(row=1, column=0, sticky="e")
        self.email_entry = tk.Entry(self.login_frame, width=30)
        self.email_entry.grid(row=1, column=1, columnspan=2, pady=5)

        tk.Label(self.login_frame, text="Password:").grid(row=2, column=0, sticky="e")
        self.password_entry = tk.Entry(self.login_frame, show="*", width=30)
        self.password_entry.grid(row=2, column=1, columnspan=2, pady=5)

        tk.Label(self.login_frame, text="User Type:").grid(row=3, column=0, sticky="e")
        self.user_type_var = tk.StringVar(value="student")
        tk.Radiobutton(self.login_frame, text="Teacher", variable=self.user_type_var, value="teacher").grid(row=3, column=1, sticky="w")
        tk.Radiobutton(self.login_frame, text="Student", variable=self.user_type_var, value="student").grid(row=3, column=2, sticky="w")

        tk.Button(self.login_frame, text="Login", command=self.login).grid(row=4, column=1, pady=10)
        tk.Button(self.login_frame, text="Sign Up", command=self.create_signup_page).grid(row=4, column=2, pady=10)
        tk.Button(self.login_frame, text="Forgot Password?", command=self.reset_password).grid(row=5, column=1, columnspan=2, pady=5)

    def create_signup_page(self):
        self.clear_frame()
        self.signup_frame = tk.Frame(self.master)
        self.signup_frame.pack(pady=20)

        tk.Label(self.signup_frame, text="Sign Up", font=("Helvetica", 16)).grid(row=0, column=1, columnspan=2, pady=10)

        tk.Label(self.signup_frame, text="Name:").grid(row=1, column=0, sticky="e")
        self.name_entry = tk.Entry(self.signup_frame, width=30)
        self.name_entry.grid(row=1, column=1, columnspan=2, pady=5)

        tk.Label(self.signup_frame, text="Email:").grid(row=2, column=0, sticky="e")
        self.signup_email_entry = tk.Entry(self.signup_frame, width=30)
        self.signup_email_entry.grid(row=2, column=1, columnspan=2, pady=5)

        tk.Label(self.signup_frame, text="Password:").grid(row=3, column=0, sticky="e")
        self.signup_password_entry = tk.Entry(self.signup_frame, show="*", width=30)
        self.signup_password_entry.grid(row=3, column=1, columnspan=2, pady=5)

        tk.Label(self.signup_frame, text="Confirm Password:").grid(row=4, column=0, sticky="e")
        self.confirm_password_entry = tk.Entry(self.signup_frame, show="*", width=30)
        self.confirm_password_entry.grid(row=4, column=1, columnspan=2, pady=5)

        tk.Label(self.signup_frame, text="User Type:").grid(row=5, column=0, sticky="e")
        self.signup_user_type_var = tk.StringVar(value="student")
        tk.Radiobutton(self.signup_frame, text="Teacher", variable=self.signup_user_type_var, value="teacher").grid(row=5, column=1, sticky="w")
        tk.Radiobutton(self.signup_frame, text="Student", variable=self.signup_user_type_var, value="student").grid(row=5, column=2, sticky="w")

        tk.Button(self.signup_frame, text="Create Account", command=self.signup).grid(row=6, column=1, pady=10)
        tk.Button(self.signup_frame, text="Back to Login", command=self.create_login_page).grid(row=6, column=2, pady=10)

    def show_student_dashboard(self):
        self.clear_frame()
        self.dashboard_frame = tk.Frame(self.master)
        self.dashboard_frame.pack(pady=20)

        tk.Label(self.dashboard_frame, text="Student Dashboard", font=("Helvetica", 16)).pack(pady=10)

        buttons = [
            ("View Assignments", self.view_assignments),
            ("View Scores", self.view_scores),
            ("View Leaderboard", self.view_leaderboard),
            ("Explore Quizzes", self.explore_quizzes)
        ]

        for text, command in buttons:
            btn = tk.Button(self.dashboard_frame, text=text, font=("Helvetica", 14), width=20, height=5, command=command)
            btn.pack(pady=10)

    def view_assignments(self):
        self.show_page("View Assignments")

    def view_scores(self):
        self.show_page("View Scores")

    def view_leaderboard(self):
        self.show_page("View Leaderboard")

    def explore_quizzes(self):
        self.show_page("Explore Quizzes")

    def show_page(self, title):
        self.clear_frame()
        page_frame = tk.Frame(self.master)
        page_frame.pack(pady=20)
        tk.Label(page_frame, text=title, font=("Helvetica", 16)).pack(pady=10)
        tk.Button(page_frame, text="Back", command=self.show_student_dashboard).pack(pady=10)

    def reset_password(self):
        email = self.email_entry.get()
        if email:
            try:
                auth.generate_password_reset_link(email)
                messagebox.showinfo("Success", "A password reset email has been sent.")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to send reset email. Please try again.\n{e}")

    def signup(self):
        name = self.name_entry.get()
        email = self.signup_email_entry.get()
        password = self.signup_password_entry.get()
        confirm_password = self.confirm_password_entry.get()
        user_type = self.signup_user_type_var.get()

        if not name or not email or not password or not confirm_password:
            messagebox.showerror("Error", "All fields are required.")
            return
        if password != confirm_password:
            messagebox.showerror("Error", "Passwords do not match.")
            return

        try:
            user = auth.create_user(email=email, password=password)
            db.collection("users").document(user.uid).set({"name": name, "email": email, "user_type": user_type})
            messagebox.showinfo("Success", f"Account created for {user_type}")
            self.create_login_page()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to create account.\n{e}")

    def login(self):
        email = self.email_entry.get()
        password = self.password_entry.get()
        self.user_type = self.user_type_var.get()

        if not email or not password:
            messagebox.showerror("Error", "All fields are required.")
            return

        try:
            user = auth.get_user_by_email(email)
            if self.user_type == "teacher":
                messagebox.showinfo("Success", "Teacher login successful!")
            else:
                self.show_student_dashboard()
        except Exception as e:
            messagebox.showerror("Error", "Login failed. Check your credentials.")

    def clear_frame(self):
        for widget in self.master.winfo_children():
            widget.destroy()

# Initialize the App
def main():
    root = tk.Tk()
    app = QuizApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()


ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.